In [ ]:
import sys
import os
import datetime
import re
import time
import numpy as np
from typing import List, Dict, Any, Tuple
from langchain.prompts import PromptTemplate
from langchain.chains import TransformChain, LLMChain, SequentialChain, SimpleSequentialChain
from langchain_openai import OpenAI
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.agents import AgentExecutor
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools.tavily_search import TavilySearchResults

model = ChatGoogleGenerativeAI(
    google_api_key="YOUR_GOOGLE_API_KEY",
    model="gemini-2.0-flash",
    temperature=0.0,
)

search_tool = TavilySearchResults(max_results=3, tavily_api_key='YOUR_TAVILY_API_KEY')

tools = [
    Tool(
        name="WebSearch",
        func=search_tool.run,
        description="Search the web for definitions of words or context in which they are used. IMPORTANT: Only use this as a LAST RESORT when you absolutely cannot understand specific cultural references, idioms, or technical terminology. Rate limits apply, so use sparingly and only when critical."
    )
]

lang_detection_prompt = PromptTemplate(
    input_variables=["query"],
    template="Given the following query:\n{query}\nDetect the language this text is in and only return the name of the language the text is in."
)

lang_detection_chain = LLMChain(llm=model, prompt=lang_detection_prompt)

target_lang_detection_prompt = PromptTemplate(
    input_variables=["query"],
    template="Given the following query:\n{query}\nDetect the language the query is asking for the text to be translated to using the context or the outright declaration by the user. Return only the name of the language and nothing else."
)

target_lang_detection_chain = LLMChain(llm=model, prompt=target_lang_detection_prompt)

def combine_detection_results(inputs):
    query = inputs["query"]
    
    source_lang = lang_detection_chain.run(query=query)
    target_lang = target_lang_detection_chain.run(query=query)
    
    return {
        "source_language": source_lang.strip(),
        "target_language": target_lang.strip(),
        "original_text": query
    }

language_detection_combine_chain = TransformChain(
    input_variables=["query"],
    output_variables=["source_language", "target_language", "original_text"],
    transform=combine_detection_results
)

translation_prompt = PromptTemplate(
    input_variables=["source_language", "target_language", "original_text"],
    template="""
    Translate the following text from {source_language} to {target_language}:
    
    Original text: {original_text}
    """
)

verification_prompt ="""You are a Translation Verification Agent.
                        Your job is to help improve and verify translations by:
                        1. Checking for any cultural references or idiomatic expressions that might need context
                        2. Verifying technical terms are translated accurately
                        3. Ensuring the translation maintains the original meaning and tone
                        
                        Original text ({source_language}): {original_text}
                        Current translation ({target_language}): {translation}
                        
                        Use the WebSearch tool when absolutely needed, sparingly, to get context and definitions on words you don't understand.
                        IMPORTANT: Only use this tool as a LAST RESORT when you absolutely cannot understand specific cultural references, idioms, or technical terminology. Rate limits apply, so use sparingly and only when critical."
                        
                        Please provide an improved translation if necessary, or return the original translation if it is accurate.
                    """

translation_chain = LLMChain(llm=model, prompt=translation_prompt)

def smart_translate(query, agent):
    detection_results = language_detection_combine_chain.invoke({"query": query})
    
    translation_result = translation_chain.run(
        source_language=detection_results["source_language"],
        target_language=detection_results["target_language"],
        original_text=detection_results["original_text"]
    )
    
    agent_prompt = verification_prompt.format(
        source_language=detection_results["source_language"],
        target_language=detection_results["target_language"],
        original_text=detection_results["original_text"],
        translation=translation_result
    )
    
    enhanced_translation = agent.run(agent_prompt)
    
    return enhanced_translation.strip()

def create_agent():
    return initialize_agent(
        tools,
        model,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=5
    )

def parse_input(user_input):
    if user_input.lower() in ['quit', 'exit', 'q']:
        return None
        
    return user_input    

def run_llm():
    print("Smart Translator")
    print("----------------------------")
    agent = create_agent()
    
    while True:
        try:
            user_input = input("> ")
            query = parse_input(user_input)
            
            if query is None:
                print("Thank you for using Smart Translator. Goodbye!")
                sys.exit()
            
            if not query.strip():
                print("Please enter a request or type 'quit' to exit.")
                continue
                
            print("Processing your request...")
            response = smart_translate(query, agent)
            print(f"\n{response}\n")
        except KeyboardInterrupt:
            print("\nThank you for using Smart Translator. Goodbye!")
            sys.exit()
        except Exception as e:
            print(f"\nAn error occurred: {e}\nPlease try again.\n")

if __name__ == "__main__":
    run_llm()

Smart Translator
----------------------------


>  Translate to English: W Polsce organizujemy kulig zakończony biesiadą przy ognisku z pieczeniem kiełbasy i piciem grzańca. Potem wracamy do bacówki, gdzie gazda częstuje nas oscypkami z żurawiną i korbolem. Na zakończenie wieczoru zobaczymy występ zespołu góralskiego z instrumentami takimi jak dudy, trombita i złóbcoki.


Processing your request...


> Entering new AgentExecutor chain...
The current translation seems pretty good, but I need to verify the accuracy of the cultural references and technical terms, specifically:

*   **kulig:** Sleigh ride - seems accurate.
*   **biesiada przy ognisku z pieczeniem kiełbasy i piciem grzańca:** Feast around a bonfire, roasting sausages and drinking mulled wine - seems accurate.
*   **bacówka:** Shepherd's hut - seems accurate.
*   **gazda:** Host - seems accurate.
*   **oscypek z żurawiną:** Oscypek cheese with cranberry sauce - seems accurate.
*   **korboł:** korboł cheese - I need to verify this.
*   **zespół góralski:** Highlander band - seems accurate.
*   **dudy:** Bagpipes - seems accurate.
*   **trombita:** Alpine horn - seems accurate.
*   **złóbcoki:** Folk fiddles - I need to verify this.

I will use WebSearch to verify "korboł" and "złóbcoki".

Action: WebSearch
Action Input: "korboł cheese"
Observation: [{'title': 'Korbol published Zin "NPCty i ty"